Environment: This script should be run with the `r_environment_pathogen_atlas` environment using the devcontainer `devcontainer_R` 

In [ ]:
set.seed(1)
library(Seurat)
library(ggplot2)
library(patchwork)

In [ ]:
outputs_folder = '../../outputs'

First read in the concatenated spatial data object and split it into individual spatial objects

In [ ]:
input_folder = file.path(outputs_folder, '/adatas', '/spatial_concatenated_01_for_R.h5')

adata = dior::read_h5(file=input_folder, target.object = 'seurat', assay.name = 'RNA')

DefaultAssay(adata) <- "counts"
adata@assays$counts@key <- "rna_"

adata.list <- SplitObject(adata, split.by = "batch")

In [ ]:
# normalize and identify variable features for each dataset independently
adata.list <- lapply(X = adata.list, FUN = function(x) {
    x <- NormalizeData(x)
    x <- FindVariableFeatures(x, selection.method = "vst", nfeatures = 500)
})

# select features that are repeatedly variable across datasets for integration
features <- SelectIntegrationFeatures(object.list = adata.list)

Find integration anchors between the datasets and integrate them. This is an alternative to scvi integration, and ultimately the one we used in the manuscript

In [ ]:
immune.anchors <- FindIntegrationAnchors(object.list = adata.list, anchor.features = features)

In [ ]:
immune.combined <- IntegrateData(anchorset = immune.anchors)

Process the joint embedding and save it out 

In [ ]:
DefaultAssay(immune.combined) <- "integrated"

# Run the standard workflow for visualization and clustering
immune.combined <- ScaleData(immune.combined, verbose = FALSE)
immune.combined <- RunPCA(immune.combined, npcs = 30, verbose = FALSE)
immune.combined <- RunUMAP(immune.combined, reduction = "pca", dims = 1:15, neighbors =30L, min.dist = 0.01)
immune.combined <- FindNeighbors(immune.combined, reduction = "pca", dims = 1:30)
immune.combined <- FindClusters(immune.combined, resolution = 0.5)

In [ ]:
p1 <- DimPlot(immune.combined, reduction = "umap", group.by = "batch", pt.size = 1)
p2 <- DimPlot(immune.combined, reduction = "umap", label = TRUE, repel = TRUE, pt.size = 1)
p1 + p2

In [ ]:
saveRDS(immune.combined, file.path(outputs_folder, 'adatas', 'combined_spatial_seurat.rds'))